# DCASE 2022 VQ-VAE + PixelSNAIL — Training on Colab

Runs the project training pipeline on Google Colab with **longer epochs**.

**Setup:** Run cells **1 → 2 → 3 → 4 → 5** in order (Mount Drive → Clone repo → Project & deps → Config → Run training). Cell **6** runs evaluation and compares ROC AUC for reconstruction vs PixelSNAIL. Set `DATA_ROOT` in the config cell to your dataset path. **Full step-by-step guide:** see `notebooks/COLAB_TRAINING_GUIDE.md` in the repo.

## 1. Mount Google Drive
Required so we can clone the project under `/content/drive/MyDrive/` and (optionally) keep dataset and checkpoints on Drive.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Clone project into Drive
Clone the repo under your Drive so the Colab kernel has the full project (including `src/`). If the folder already exists, we pull the latest; otherwise we clone. Change `REPO_DIR` if you want a different path.

In [6]:
import os

# Clone into this folder under your Drive (My Drive = /content/drive/MyDrive)
REPO_DIR = "/content/drive/MyDrive/semcom_asd_vqar"
REPO_URL = "https://github.com/raidantimosquitos/semcom_asd_vqar.git"

if os.path.isdir(REPO_DIR):
    !cd "{REPO_DIR}" && git pull
else:
    os.makedirs(os.path.dirname(REPO_DIR), exist_ok=True)
    !git clone {REPO_URL} {REPO_DIR}

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 7 (delta 5), reused 7 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (7/7), 4.25 KiB | 9.00 KiB/s, done.
From https://github.com/raidantimosquitos/semcom_asd_vqar
   5ac8692..e40ed10  main       -> origin/main
Updating 5ac8692..e40ed10
Fast-forward
 notebooks/colab_train.ipynb | 925 ++++++++++++++++++++++++++++++++++++++++----
 src/engine/test.py          |  12 +-
 2 files changed, 864 insertions(+), 73 deletions(-)


## 3. Project and dependencies
**Run this cell before any training.** Uses `PROJECT_ROOT` from the clone path above, adds it to `sys.path`, and installs deps so `src` can be imported.

In [3]:
import sys
import os

# Use the clone path from the cell above (or set manually if you cloned elsewhere)
try:
    PROJECT_ROOT = REPO_DIR
except NameError:
    PROJECT_ROOT = "/content/drive/MyDrive/semcom_asd_vqar"

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

# Install dependencies (Colab usually has torch/numpy/sklearn; add PyYAML if missing)
!pip install -q PyYAML

## 4. Config and paths
Set `DATA_ROOT` to your dataset (e.g. on Drive: `/content/drive/MyDrive/datasets/dcase2020-task2-dev-dataset`). Checkpoints and logs are under the cloned project on Drive.

In [4]:
# Dataset path: on Colab use Drive path after mount, e.g. "/content/drive/MyDrive/datasets/dcase2020-task2-dev-dataset"
DATA_ROOT = "/content/drive/MyDrive/datasets/dcase2020-task2-dev-dataset"

# Checkpoints and logs under the cloned project on Drive
CHECKPOINT_DIR = "./checkpoints"
LOG_DIR = "./logs"

# Overrides: longer epochs (merge into config)
OVERRIDES = {
    "data": {"root_dir": DATA_ROOT},
    "checkpoints": {"dir": CHECKPOINT_DIR},
    "phase1": {
        "num_epochs": 15,
        "lr": 0.002,
        "checkpoint": f"{CHECKPOINT_DIR}/models/mobilenetv2_8x_vqvae.pth",
    },
    "phase2": {
        "num_epochs": 30,
        "lr": 0.0002,
        "checkpoint": f"{CHECKPOINT_DIR}/models/pixelsnail_prior.pth",
    },
    "eval": {
        "vqvae_checkpoint": f"{CHECKPOINT_DIR}/models/mobilenetv2_8x_vqvae.pth",
        "prior_checkpoint": f"{CHECKPOINT_DIR}/models/pixelsnail_prior.pth",
    },
    "logging": {"log_dir": LOG_DIR},
}

os.makedirs(f"{CHECKPOINT_DIR}/models", exist_ok=True)  
os.makedirs(LOG_DIR, exist_ok=True)

## 5. Run training
**Run cells 1–4 first.** Uses `configs/colab.yaml` and the overrides above. Logs go to the logger (console + file).

In [5]:
# Sanity check: run cells 1–4 first (Mount, Clone, Dependencies, Config)
assert "PROJECT_ROOT" in dir(), "Run the 'Project and dependencies' cell (section 3) first."
from src.main import run

run(
    config_path=os.path.join(PROJECT_ROOT, "configs", "colab.yaml"),
    overrides=OVERRIDES,
    mode="train",
    log_dir=LOG_DIR,
)

2026-02-18 06:53:49 | INFO | Config: {'mode': 'train', 'checkpoints': {'dir': './checkpoints'}, 'data': {'root_dir': '/content/drive/MyDrive/datasets/dcase2020-task2-dev-dataset', 'appliance': 'fan', 'test_size': 0.1, 'batch_size': 32, 'random_state': 42, 'max_samples_stats': None}, 'device': 'cuda', 'phase1': {'num_epochs': 15, 'lr': 0.002, 'checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth'}, 'phase2': {'num_epochs': 30, 'lr': 0.0002, 'checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'eval': {'vqvae_checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth', 'prior_checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'logging': {'log_dir': './logs', 'name': 'main'}}


INFO:main:Config: {'mode': 'train', 'checkpoints': {'dir': './checkpoints'}, 'data': {'root_dir': '/content/drive/MyDrive/datasets/dcase2020-task2-dev-dataset', 'appliance': 'fan', 'test_size': 0.1, 'batch_size': 32, 'random_state': 42, 'max_samples_stats': None}, 'device': 'cuda', 'phase1': {'num_epochs': 15, 'lr': 0.002, 'checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth'}, 'phase2': {'num_epochs': 30, 'lr': 0.0002, 'checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'eval': {'vqvae_checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth', 'prior_checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'logging': {'log_dir': './logs', 'name': 'main'}}


2026-02-18 06:53:49 | INFO | Mode: train


INFO:main:Mode: train


2026-02-18 06:53:49 | INFO | Config: {'mode': 'train', 'checkpoints': {'dir': './checkpoints'}, 'data': {'root_dir': '/content/drive/MyDrive/datasets/dcase2020-task2-dev-dataset', 'appliance': 'fan', 'test_size': 0.1, 'batch_size': 32, 'random_state': 42, 'max_samples_stats': None}, 'device': 'cuda', 'phase1': {'num_epochs': 15, 'lr': 0.002, 'checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth'}, 'phase2': {'num_epochs': 30, 'lr': 0.0002, 'checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'eval': {'vqvae_checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth', 'prior_checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'logging': {'log_dir': './logs', 'name': 'main'}}


...

INFO:main:Prior Epoch 29/30 | train_nll: 2.079482 | val_nll: 2.084490


2026-02-18 09:01:21 | INFO | Prior Epoch 30/30 | train_nll: 2.075107 | val_nll: 2.081413 [best]


INFO:main:Prior Epoch 30/30 | train_nll: 2.075107 | val_nll: 2.081413 [best]


2026-02-18 09:01:21 | INFO | Prior training finished. best_val_nll: 2.081413 | checkpoint: ./checkpoints/models/pixelsnail_prior.pth


INFO:main:Prior training finished. best_val_nll: 2.081413 | checkpoint: ./checkpoints/models/pixelsnail_prior.pth


## 6. Evaluate — ROC AUC comparison

Run evaluation on the test set and compare **ROC AUC** for the two anomaly criteria (same as `src.engine.test` and `python -m src.main --config configs/colab.yaml --mode eval`):

1. **Reconstruction (MSE)** — higher reconstruction error = more anomalous  
2. **PixelSNAIL NLL** — higher negative log-likelihood on VQ codes = more anomalous  

Uses the same config and checkpoints as above (from the Config cell). Run **after** training (or if you already have checkpoints).

In [7]:
from pathlib import Path
from src.utils.config import load_config, deep_merge
from src.utils.logger import get_logger, log_config
from src.engine.test import run_evaluation

# Same config as training (follows test module and main program)
config = load_config(os.path.join(PROJECT_ROOT, "configs", "colab.yaml"))
deep_merge(config, OVERRIDES)
config["mode"] = "eval"

log_path = Path(LOG_DIR) / "main_eval.log"
log_path.parent.mkdir(parents=True, exist_ok=True)
logger = get_logger("main", log_file=log_path)
log_config(logger, config=config)
logger.info("Mode: eval")

results = run_evaluation(config, logger)

# Compare ROC AUC for the two anomaly criteria
if results:
    print("\n" + "=" * 52)
    print("  ROC AUC comparison (higher = better anomaly detection)")
    print("=" * 52)
    print(f"  Criterion              | ROC AUC")
    print("-" * 52)
    print(f"  Reconstruction (MSE)    | {results['auc_mse']:.4f}")
    print(f"  PixelSNAIL NLL         | {results['auc_nll']:.4f}")
    print("=" * 52)
    print(f"  Test samples: {results['num_test_samples']}")
    better = "PixelSNAIL NLL" if results["auc_nll"] >= results["auc_mse"] else "Reconstruction (MSE)"
    print(f"  Better criterion: {better}")

2026-02-18 09:20:46 | INFO | Config: {'mode': 'eval', 'checkpoints': {'dir': './checkpoints'}, 'data': {'root_dir': '/content/drive/MyDrive/datasets/dcase2020-task2-dev-dataset', 'appliance': 'fan', 'test_size': 0.1, 'batch_size': 32, 'random_state': 42, 'max_samples_stats': None}, 'device': 'cuda', 'phase1': {'num_epochs': 15, 'lr': 0.002, 'checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth'}, 'phase2': {'num_epochs': 30, 'lr': 0.0002, 'checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'eval': {'vqvae_checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth', 'prior_checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'logging': {'log_dir': './logs', 'name': 'main'}}


INFO:main:Config: {'mode': 'eval', 'checkpoints': {'dir': './checkpoints'}, 'data': {'root_dir': '/content/drive/MyDrive/datasets/dcase2020-task2-dev-dataset', 'appliance': 'fan', 'test_size': 0.1, 'batch_size': 32, 'random_state': 42, 'max_samples_stats': None}, 'device': 'cuda', 'phase1': {'num_epochs': 15, 'lr': 0.002, 'checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth'}, 'phase2': {'num_epochs': 30, 'lr': 0.0002, 'checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'eval': {'vqvae_checkpoint': './checkpoints/models/mobilenetv2_8x_vqvae.pth', 'prior_checkpoint': './checkpoints/models/pixelsnail_prior.pth'}, 'logging': {'log_dir': './logs', 'name': 'main'}}


2026-02-18 09:20:46 | INFO | Mode: eval


INFO:main:Mode: eval


2026-02-18 09:20:46 | INFO | Evaluation config: root_dir=/content/drive/MyDrive/datasets/dcase2020-task2-dev-dataset, appliance=fan


...

INFO:main:  prior_checkpoint: /content/drive/MyDrive/semcom_asd_vqar/checkpoints/models/pixelsnail_prior.pth


2026-02-18 09:32:57 | INFO |   ROC AUC (MSE criterion):  0.5542


INFO:main:  ROC AUC (MSE criterion):  0.5542


2026-02-18 09:32:57 | INFO |   ROC AUC (PixelSNAIL NLL): 0.5624


INFO:main:  ROC AUC (PixelSNAIL NLL): 0.5624
